In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from owi_data_2_pandas.io import API
import datetime
from pytz import utc
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

import dotenv
dotenv.load_dotenv()

True

In [3]:
loc = 'nw2'
turbine = 'c02'
mode = 'SS2'

In [4]:
# GET THE DATA
package_folder = os.path.dirname(os.path.dirname(os.getcwd()))
data_folder = os.path.join(package_folder, 'data')

In [5]:
user = os.getenv('API_USERNAME') 
password = os.getenv('API_PASSWORD')
root = r"http://10.0.0.138/api/v1/"

In [6]:
from owi_data_2_pandas.io import API
owi_api = API(api_root=root, username=user, password=password)

In [7]:
def get_big_data(start, end, owi_api, location, metrics, days=15):
    new_start = start
    all_data = pd.DataFrame()
    middle = start + datetime.timedelta(days=days)
    while end > middle:
        data_ = owi_api.query((new_start, middle), location=location, metrics = metrics)
        new_start = middle
        middle = new_start + datetime.timedelta(days=days)
        all_data = pd.concat([all_data, data_])
    if end < middle:
        data_ = owi_api.query((new_start, end), location=location, metrics = metrics)
        all_data = pd.concat([all_data, data_])
    duplicated_indices = all_data.index.duplicated(keep='first')
    all_data_unique = all_data[~duplicated_indices]
    return all_data_unique

In [18]:
start = datetime.datetime(2023, 6, 1, tzinfo=utc)
stop = datetime.datetime(2023, 12, 1, tzinfo=utc)
location = 'nw2c02'
metrics = owi_api.metrics(locations=location)
test_data = get_big_data(start, stop, owi_api, location, metrics.metric, days=10)

In [19]:
test_data_path = '_'.join([loc, turbine, mode, 'test_data.parquet'])
test_data.to_parquet(os.path.join(data_folder,'raw', loc, test_data_path), compression='gzip')

In [20]:
os.path.join(data_folder,'raw', loc, test_data_path)

'c:\\Users\\Max\\Documents\\OWI_LAB\\Code\\DCE_EOVnorm\\dce_eov_normalisation\\data\\raw\\nw2\\nw2_c02_SS2_test_data.parquet'

In [21]:
test_data.head()

,location,mean_NW2_C02_rpm,mean_NW2_C02_yaw,std_NW2_C02_pitch,mean_NW2_C02_pitch,mean_NW2_C02_power,std_NW2_C02_windspeed,mean_NW2_C02_windspeed,mean_NW2_C02_winddirection,mpe_NW2_C02_FA,...,min_NW2_C02_NAC_ACC_SS,rms_NW2_C02_NAC_ACC_FA,rms_NW2_C02_NAC_ACC_SS,std_NW2_C02_NAC_ACC_FA,std_NW2_C02_NAC_ACC_SS,mean_NW2_C02_NAC_ACC_FA,mean_NW2_C02_NAC_ACC_SS,rms1p_NW2_C02_NAC_ACC_Z,rms1p_NW2_C02_NAC_ACC_FA,rms1p_NW2_C02_NAC_ACC_SS
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-06-01 00:00:00+00:00,NW2C02,10.445,16.338,0.831241,-3.525,8118.208,0.754,11.391,17.150,"[{'size': 32, 'algorithm': 'lscf', 'std_dampin...",...,-0.036635,0.010305,0.011513,0.010305,0.011513,0.0,0.0,0.000431,0.003753,0.004629
2023-06-01 00:10:00+00:00,NW2C02,10.445,20.535,1.015521,-2.854,6843.520,1.023,10.609,22.353,"[{'size': 16, 'algorithm': 'lscf', 'std_dampin...",...,-0.033817,0.009921,0.010211,0.009921,0.010211,0.0,0.0,0.000430,0.003918,0.003613
2023-06-01 00:20:00+00:00,NW2C02,10.445,24.843,0.709950,-2.070,5771.776,1.277,10.180,24.032,"[{'size': 28, 'algorithm': 'lscf', 'std_dampin...",...,-0.036501,0.010950,0.010467,0.010950,0.010467,0.0,0.0,0.000427,0.003839,0.004551
2023-06-01 00:30:00+00:00,NW2C02,10.445,25.179,0.956972,-2.826,6920.320,1.156,11.129,24.060,"[{'size': 33, 'algorithm': 'lscf', 'std_dampin...",...,-0.038860,0.011446,0.011084,0.011446,0.011084,0.0,0.0,0.000452,0.004023,0.004891
2023-06-01 00:40:00+00:00,NW2C02,10.445,24.955,0.644423,-2.042,5688.768,0.922,10.141,25.095,"[{'size': 41, 'algorithm': 'lscf', 'std_dampin...",...,-0.042127,0.011552,0.011688,0.011552,0.011688,0.0,0.0,0.000444,0.003902,0.004729


In [22]:
from vubmvbc.config import Credentials
from vubmvbc.client import Base
from vubmvbc.objects import Catalog, Data
import vubmvbc.data_getter as dg

In [23]:
mvbc_username = os.getenv('MEETNET_USERNAME') # Replace with your usernam
mvbc_password = os.getenv('MEETNET_PASSWORD') # Replace with your password
creds = Credentials(username=mvbc_username, password=mvbc_password)
b=Base(creds)
b.ping()

c = Catalog(credentials=creds)
df_unfiltered = c.data_points()

start  = datetime.datetime(2023,6,1,tzinfo=utc)
stop  = datetime.datetime(2023,12,1,tzinfo=utc)

westhinder_weather_station = 'Westhinder'
westhinder_weather_station_data = \
    dg.get_data_by_weatherstation(
        westhinder_weather_station,
        start,
        stop,
        creds,
        df_unfiltered
    )
westhinder_weather_station_data = westhinder_weather_station_data.resample('10T', axis=0).interpolate(method='linear', axis=0, limit=12)

wandelaar_weather_station = 'Wandelaar'
wandelaar_weather_station_data = \
    dg.get_data_by_weatherstation(
        wandelaar_weather_station,
        start,
        stop,
        creds,
        df_unfiltered
    )
wandelaar_weather_station_data = wandelaar_weather_station_data.resample('10T', axis=0).interpolate(method='linear', axis=0, limit=12)

c:\Users\Max\anaconda3\envs\dce_eov_norm\Lib\site-packages\vubmvbc\data_getter.py:47: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df["Name"][df.index == id_][0].replace(" - ", "").replace(" ", ""),
c:\Users\Max\anaconda3\envs\dce_eov_norm\Lib\site-packages\vubmvbc\data_getter.py:48: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df["ParameterName"][df.index == id_][0]
c:\Users\Max\anaconda3\envs\dce_eov_norm\Lib\site-packages\vubmvbc\data_getter.py:47: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFr

In [24]:
wandelaar_weather_station_data.isna().sum()

mvbc_WandelaarBuoy_10%_highest_waves                                        0
mvbc_WandelaarBuoy_Wave_height                                              0
mvbc_WandelaarBuoy_Average_wave_period                                      0
mvbc_WandelaarBuoy_Height_waves_with_period_>_10_s                          0
mvbc_WandelaarBuoy_Sea_water_temperature                                    0
mvbc_WandelaarMeasuringpile_Max_3-seconds_wind_gust_(at_10_m_height)        0
mvbc_WandelaarMeasuringpile_Average_wind_direction                          0
mvbc_WandelaarMeasuringpile_Average_wind_speed_(at_10_m_height)             0
mvbc_WandelaarMeasuringpile_Tide_TAW                                    25850
mvbc_WandelaarMeasuringpile_Relative_humidity                               2
mvbc_WandelaarMeasuringpile_Air_pressure                                    2
mvbc_WandelaarMeasuringpile_Air_temperature                                 0
mvbc_WandelaarBuoy_1%_wave_height                               

In [25]:
westhinder_weather_station_data.isna().sum()

mvbc_WesthinderBuoy_10%_highest_waves                                    436
mvbc_WesthinderBuoy_Wave_height                                          436
mvbc_WesthinderBuoy_Average_wave_period                                  436
mvbc_WesthinderBuoy_Height_waves_with_period_>_10_s                      436
mvbc_WesthinderBuoy_High_frequent_wave_direction                         436
mvbc_WesthinderBuoy_Low_frequent_wave_direction                          436
mvbc_WesthinderMeasuringpile_Sea_water_temperature                       741
mvbc_WesthinderBuoy_Sea_water_temperature                                436
mvbc_WesthinderMeasuringpile_Max_3-seconds_wind_gust_(at_10_m_height)    436
mvbc_WesthinderMeasuringpile_Average_wind_direction                      450
mvbc_WesthinderMeasuringpile_Average_wind_speed_(at_10_m_height)         436
mvbc_WesthinderMeasuringpile_Tide_TAW                                    436
mvbc_WesthinderMeasuringpile_Relative_humidity                           456

In [26]:
wandelaar_weather_station_data.to_parquet(os.path.join(data_folder,'external', loc, 'test_wandelaar_weather_data.parquet'), compression='gzip')
westhinder_weather_station_data.to_parquet(os.path.join(data_folder,'external', loc, 'test_westhinder_weather_data.parquet'), compression='gzip')